In [1]:
import pandas as pd
import seaborn as sns
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
#import pandas_profiling
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import sklearn.feature_extraction.text as text
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import warnings
!pip install contractions
import contractions
from collections import Counter
!pip install wordcloud
from wordcloud import WordCloud
from sklearn. preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import nltk
nltk.download('stopwords')
import numpy as np
warnings.filterwarnings('ignore')
import gensim
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gruhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gruhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_json('data.json',lines = True)

In [3]:
data['category'].unique()

array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT', 'EDUCATION', 'CRIME', 'SCIENCE', 'WELLNESS',
       'BUSINESS', 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA',
       'QUEER VOICES', 'HOME & LIVING', 'WOMEN', 'BLACK VOICES', 'TRAVEL',
       'MONEY', 'RELIGION', 'LATINO VOICES', 'IMPACT', 'WEDDINGS',
       'COLLEGE', 'PARENTS', 'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE',
       'HEALTHY LIVING', 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST',
       'FIFTY', 'ARTS', 'DIVORCE'], dtype=object)

In [4]:
new_mappings = {tuple(['U.S. NEWS', 'WORLD NEWS',  'GOOD NEWS' , 'THE WORLDPOST','WORLDPOST']): 'NEWS',
                  tuple(['CULTURE & ARTS', 'TRAVEL','ARTS & CULTURE','ARTS']): 'TRAVEL AND ARTS',
                                                  tuple(['WELLNESS','STYLE & BEAUTY','HOME & LIVING','HEALTHY LIVING','STYLE']): 'LIFESTYLE',
                                                 tuple(['TASTE','FOOD & DRINK']): 'FOOD',
                                                  tuple(['QUEER VOICES', 'BLACK VOICES','LATINO VOICES','WOMEN']): 'DIVERSITY',
                                                  tuple(['COMEDY','SPORTS','ENTERTAINMENT','WEIRD NEWS']): 'ENTERTAINMENT',
                                                  tuple(['BUSINESS','MONEY']): 'BUSINESS',
                                                  tuple(['COLLEGE','PARENTING','EDUCATION','PARENTS']): 'PARENTING AND EDUCATION',
                                                  tuple(['CRIME','FIFTY','IMPACT','WEDDINGS','DIVORCE','RELIGION']): 'MISCELLANEOUS',
                                                tuple(['POLITICS','MEDIA']): 'POLITICS',
                                                 tuple(['ENVIRONMENT','GREEN']): 'ENVIRONMENT',
                                                 tuple(['TECH','SCIENCE']): 'TECHNOLOGY'
                }

In [5]:
new_mapping_dict = {}
for key, value in new_mappings.items():
    for one_key in key:
        new_mapping_dict[one_key] = value

In [6]:
data['new_category'] = data['category'].replace(new_mapping_dict)
data.loc[:,['category','new_category']]

,category,new_category
0,U.S. NEWS,NEWS
1,U.S. NEWS,NEWS
2,COMEDY,ENTERTAINMENT
3,PARENTING,PARENTING AND EDUCATION
4,U.S. NEWS,NEWS
...,...,...
209522,TECH,TECHNOLOGY
209523,SPORTS,ENTERTAINMENT
209524,SPORTS,ENTERTAINMENT
209525,SPORTS,ENTERTAINMENT


In [7]:
data.new_category.unique()

array(['NEWS', 'ENTERTAINMENT', 'PARENTING AND EDUCATION',
       'TRAVEL AND ARTS', 'TECHNOLOGY', 'POLITICS', 'ENVIRONMENT',
       'MISCELLANEOUS', 'LIFESTYLE', 'BUSINESS', 'FOOD', 'DIVERSITY'],
      dtype=object)

In [8]:
data.drop_duplicates(inplace = True)

In [9]:
data.drop_duplicates(subset=['headline', 'short_description'], inplace = True)

In [10]:
data.shape

(209038, 7)

In [11]:
data.isnull().values.any()

False

In [12]:
df = data

In [13]:
df['all_words'] = df['headline']+" "+df['short_description']

In [14]:
df['all_words'][0]

'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.'

# data Cleaning / preprocessing

In [15]:
def expand_contractions(combined_words):
    expanded_words = []   
    for word in combined_words.split():
        expanded_words.append(contractions.fix(word))  
    expanded_text = ' '.join(expanded_words)
    return expanded_text

In [16]:
stop_words = set(stopwords.words('english'))

In [17]:
def remove_stop_words(x):
    lst = [i for i in x.split() if not i in stop_words]
    data = ' '.join(lst)
    return data

In [18]:
def clean_data(data):
    data = str(data)
    data = data.lower()
    data = expand_contractions(data)
    data = re.sub('@[A-Za-z0-9_]+',' ', data)
    data = re.sub('#[A-Za-z0-9_]+',' ', data)
    data = re.sub('(http\S+)', ' ', data)
    data = re.sub('www.\S+', ' ', data)
    data = re.sub('[()!?]', ' ', data)
    data = re.sub('\[.*?\]',' ', data)
    data = re.sub('[^a-z0-9]',' ', data)
    data = re.sub(' +',' ', data)
    data = remove_stop_words(data)
    return data

In [19]:
clean_df = df.loc[:,['all_words','new_category']]

In [20]:
clean_df['clean_all_words'] = clean_df['all_words'].apply(clean_data)

In [21]:
clean_df

,all_words,new_category,clean_all_words
0,Over 4 Million Americans Roll Up Sleeves For O...,NEWS,4 million americans roll sleeves omicron targe...
1,"American Airlines Flyer Charged, Banned For Li...",NEWS,american airlines flyer charged banned life pu...
2,23 Of The Funniest Tweets About Cats And Dogs ...,ENTERTAINMENT,23 funniest tweets cats dogs week sept 17 23 d...
3,The Funniest Tweets From Parents This Week (Se...,PARENTING AND EDUCATION,funniest tweets parents week sept 17 23 accide...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,NEWS,woman called cops black bird watcher loses law...
...,...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECHNOLOGY,rim ceo thorsten heins significant plans black...
209523,Maria Sharapova Stunned By Victoria Azarenka I...,ENTERTAINMENT,maria sharapova stunned victoria azarenka aust...
209524,"Giants Over Patriots, Jets Over Colts Among M...",ENTERTAINMENT,giants patriots jets colts among improbable su...
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,ENTERTAINMENT,aldon smith arrested 49ers linebacker busted d...


# Word2Vec

In [22]:
w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [23]:
print(w2v_model.most_similar('delicious'))

[('scrumptious', 0.8770448565483093), ('tasty', 0.8730388879776001), ('delectable', 0.8363204598426819), ('yummy', 0.8119936585426331), ('flavorful', 0.7428319454193115), ('sinfully_delicious', 0.7334188222885132), ('scrumptious_desserts', 0.7285338640213013), ('delish', 0.7256091833114624), ('savory', 0.7186153531074524), ('decadent_dessert', 0.7114437818527222)]


In [24]:
print(w2v_model.similarity('Scientist','Researcher'))
print(w2v_model.similarity('Lake','River'))
print(w2v_model.similarity('Lake','Iphone'))

0.65890074
0.63959724
-0.052412063


In [25]:
all_sentences = clean_df['clean_all_words'].tolist()

In [26]:
len(w2v_model.get_vector('tiger'))


300

In [27]:
w2v_model.get_vector('tiger')

array([-6.83593750e-02,  1.82617188e-01, -1.77734375e-01,  1.66015625e-01,
        1.46484375e-01,  2.53906250e-02,  5.00488281e-02, -3.51562500e-02,
        2.53906250e-01,  2.22778320e-03, -3.26156616e-04, -2.05078125e-01,
       -2.98828125e-01, -3.71093750e-01, -4.45312500e-01, -2.08007812e-01,
       -1.89453125e-01, -5.72204590e-04, -2.45117188e-01, -2.08984375e-01,
        2.51953125e-01, -3.73046875e-01,  1.91406250e-01, -2.07519531e-02,
       -2.96875000e-01,  1.24023438e-01, -4.60937500e-01,  1.42578125e-01,
        7.14843750e-01,  1.18652344e-01, -1.47460938e-01, -1.15234375e-01,
       -9.03320312e-02, -8.49609375e-02, -3.14453125e-01,  1.54296875e-01,
       -1.97265625e-01,  2.89062500e-01,  3.26171875e-01,  2.79296875e-01,
       -2.13623047e-02, -7.76367188e-02,  1.29882812e-01,  1.79687500e-01,
        2.85156250e-01, -2.45117188e-01, -7.22656250e-02, -1.08398438e-01,
        3.96484375e-01,  2.75390625e-01, -2.94921875e-01,  2.87109375e-01,
       -9.33837891e-03, -

In [28]:
vocab = set(w2v_model.key_to_index.keys())

In [29]:
'tiger' in vocab

True

### Creating Average Word2Vec

In [30]:
def get_avg_w2v(sentence):
    current_vect = np.zeros(300)
    cnt_of_words = 0
    for word in sentence.split():
        if word in vocab:
            vector = w2v_model.get_vector(word)
            current_vect += vector
            cnt_of_words += 1
    current_vect /= cnt_of_words
    return current_vect


In [31]:
clean_df['avg_w2v'] = clean_df['clean_all_words'].apply(get_avg_w2v)


In [32]:
corpus = clean_df['clean_all_words'].tolist()
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

In [33]:
dic = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))

In [34]:
def get_tf_idf_w2v(sentence):
    current_vect = np.zeros(300)
    sum_of_weights = 0
    for word in sentence.split():
        try:
            vector = w2v_model.get_vector(word)
            tf_idf = dic[word]
            current_vect += (vector*tf_idf) 
            sum_of_weights += tf_idf
        except:
            continue
    if sum_of_weights > 0:
        return (current_vect/sum_of_weights)
    else:
        return np.zeros(300)
    
    

In [35]:
clean_df['tf_idf_w2v'] = clean_df['clean_all_words'].apply(get_tf_idf_w2v)

In [36]:
clean_df.shape

(209038, 5)

In [37]:
clean_df = clean_df.dropna()

In [38]:
def check_array(array):
    return np.any(np.isnan(array))

In [39]:
clean_df['bool_avg'] = clean_df['avg_w2v'].apply(check_array)

In [40]:
clean_df.loc[clean_df.bool_avg, :]

,all_words,new_category,clean_all_words,avg_w2v,tf_idf_w2v,bool_avg
63714,"""ManScraping""",ENTERTAINMENT,manscraping,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
66203,Wafflewich,FOOD,wafflewich,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
76593,The Opinionator,ENTERTAINMENT,opinionator,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
86508,Once.,ENTERTAINMENT,,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
90944,,POLITICS,,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
96824,WHAAAAAAAT!?,ENTERTAINMENT,whaaaaaaat,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
98808,"No, No, No!",ENTERTAINMENT,,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
101055,WHO Are You Now ?,ENTERTAINMENT,,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
104914,WHERE'S VLAD?,NEWS,vlad,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True
105315,#Anxiety,LIFESTYLE,,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True


In [41]:
label = LabelEncoder()
clean_df['new_category'] = label.fit_transform(clean_df['new_category'])
clean_df = clean_df.loc[~clean_df.bool_avg,['new_category','avg_w2v','tf_idf_w2v']]


#  Avg w2v Split and Classification

In [42]:
X = clean_df['avg_w2v'].to_list()
y = clean_df['new_category'].to_list()

In [43]:
x_train,x_test, y_train,y_test = train_test_split(X,Y)

NameError: name 'Y' is not defined

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd =  SGDClassifier(loss='hinge', penalty='l2')
sgd.fit(x_train,y_train)
y_pred = sgd.predict(x_test)
print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
p = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
p.fit(x_train,y_train) 
print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

# Tf - Idf W2v

In [ ]:
X = clean_df['tf_idf_w2v'].to_list()
y = clean_df['new_category'].to_list()

In [ ]:
x_train,x_test, y_train,y_test = train_test_split(X,Y)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd =  SGDClassifier(loss='hinge', penalty='l2')
sgd.fit(x_train,y_train)
y_pred = sgd.predict(x_test)
print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
p = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
p.fit(x_train,y_train) 
print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))